In [1]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,make_scorer
from sklearn import model_selection, preprocessing, ensemble
from sklearn import svm


//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df_test = pandas.read_json("final_train.json")
df_train = pandas.read_json("final_train.json")


categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if df_train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df_train[f].values) + list(df_test[f].values))
            df_train[f] = lbl.transform(list(df_train[f].values))
            df_test[f] = lbl.transform(list(df_test[f].values))



In [3]:
exclude = ['price','n_no_photo','n_expensive']
feats_to_train = [x for x in df_train.columns.tolist() if df_train[x].dtype not in ['O', '<M8[ns]'] and x not in exclude]


In [4]:
train_X = df_train[feats_to_train]
train_y=df_train['price']

In [5]:
num_map = {'high':0, 'medium':1, 'low':2}

df_train_sample = df_train.sample(frac=0.1, replace=False) 

train_X = df_train_sample[feats_to_train]
train_y = df_train_sample['interest_level'].apply(lambda x: num_map[x])
test_X = df_train_sample[feats_to_train]


In [6]:
# grid_para_svm = {'C': sp_randint(1,1000), 'kernel': ['poly'], 'degree': sp_randint(1,4)}


# grid_para_svm =  {'C': sp_randint(1,1000), 
#                   'gamma': np.arange(.0001, .0011, .0001).tolist(), 
#                   'kernel': ['rbf'], 'decision_function_shape':['ovr','ovo']}

# svm_model = svm.SVC(probability=True)
# grid_search_svm = RandomizedSearchCV(svm_model,grid_para_svm,cv=5,scoring = 'neg_log_loss',verbose=100)

# grid_search_svm.fit(train_X, train_y)

In [7]:
# print grid_search_svm.best_params_,grid_search_svm.best_score_

# {'kernel': 'rbf', 'C': 1, 'decision_function_shape': 'ovr', 'gamma': 0.001} -0.792440919858
#{'kernel': 'rbf', 'C': 381, 'decision_function_shape': 'ovr', 'gamma': 0.0009000000000000001} -0.792443231806


In [ ]:
train_X = df_train[feats_to_train]
train_y = df_train['interest_level'].apply(lambda x: num_map[x])


svm_model = svm.SVC(probability=True, kernel='rbf',C=38, decision_function_shape='ovr', gamma=0.0009,verbose=100)
svm = svm_model.fit(train_X,train_y)


preds= svm_model.predict_proba(test_X)


[LibSVM]

In [ ]:
out_df = pandas.DataFrame(preds)
out_df = pandas.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = df_test.listing_id.values


In [ ]:
out_df.to_csv("svm.csv", index=False)